In [481]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT_v, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 2개 파일을 불러왔습니다: 1201.csv, 89.csv
📊 데이터 Shape: I=2, T=24, S=5
✅ 시뮬레이션 초기화 완료: S=5, Randomness='high', M1=347.40, M2=573.92


/Users/jangseohyun/Documents/workspace/symply/DER/functions.py:96: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")


In [482]:
# models
only = gp.Model("only")
only.Params.OutputFlag = 0
only.setParam("MIPGap", 1e-7)

x = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
zeta = only.addVars(I, T, S, vtype=GRB.BINARY, name="z")
only.update()

obj = gp.quicksum(P_DA[t] * x[i, t] for i in range(I) for t in range(T)) + gp.quicksum(
    1/S * (P_RT_v[t, s] * yp[i, t, s] - P_PN[t] * ym[i, t, s]) for i in range(I) for t in range(T) for s in range(S))
only.setObjective(obj, GRB.MAXIMIZE)

for i in range(I):
    for t in range(T):
        for s in range(S):
            only.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s])
            only.addConstr(yp[i, t, s] <= R[i, t, s])
            only.addConstr(yp[i, t, s] <= M1 * zeta[i, t, s])
            only.addConstr(ym[i, t, s] <= M1 * (1 - zeta[i, t, s]))

only.optimize()

if only.status == GRB.OPTIMAL:
    print(f"[ONLY] {only.objVal}")
else:
    print("No optimal solution found.")
    
# ----------------------------------

agg = gp.Model("agg")
agg.Params.OutputFlag = 0
agg.setParam("MIPGap", 1e-7)

a = agg.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
bp = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
mu = agg.addVars(T, S, vtype=GRB.BINARY, name="z")
agg.update()

obj = gp.quicksum(P_DA[t] * a[t] for t in range(T)) + gp.quicksum(
    1/S * (P_RT_v[t, s] * bp[t, s] - P_PN[t] * bm[t, s]) for t in range(T) for s in range(S))
agg.setObjective(obj, GRB.MAXIMIZE)

for t in range(T):
    for s in range(S):
        agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - a[t] == bp[t, s] - bm[t, s])
        agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) >= bp[t, s])
        agg.addConstr(bp[t, s] <= M2 * mu[t, s])
        agg.addConstr(bm[t, s] <= M2 * (1 - mu[t, s]))
agg.optimize()

if agg.status == GRB.OPTIMAL:
    print(f"[AGG] {agg.objVal}")
else:
    print("No optimal solution found.")

[ONLY] 400642.74895722815
[AGG] 403336.12988487654


In [483]:
x_v   = np.array([[only.getVarByName(f"x[{i},{t}]").X for t in range(T)] for i in range(I)])
yp_v = np.array([[[only.getVarByName(f"y_plus[{i},{t},{s}]").X for s in range(S)] for t in range(T)] for i in range(I)])
ym_v = np.array([[[only.getVarByName(f"y_minus[{i},{t},{s}]").X for s in range(S)] for t in range(T)] for i in range(I)])
a_v  = np.array([agg.getVarByName(f"alpha[{t}]").X for t in range(T)])
bp_v = np.array([[agg.getVarByName(f"beta_plus[{t},{s}]").X for s in range(S)] for t in range(T)])
bm_v = np.array([[agg.getVarByName(f"beta_minus[{t},{s}]").X for s in range(S)] for t in range(T)])

idx = 3

xv   = x_v
ypv  = yp_v[:, :, idx]
ymv  = ym_v[:, :, idx]
av   = a_v
bpv  = bp_v[:, idx]
bmv  = bm_v[:, idx]
P_RT = P_RT_v[:, idx]


In [484]:
delta_DA = np.zeros(T)
delta_BMp = np.zeros(T)
delta_BMm = np.zeros(T)
lambd = np.zeros((I, T))
total_surplus = np.zeros(T)

for t in range(T):
    delta_DA[t]  = av[t]  - np.sum(xv[:, t])
    delta_BMp[t] = bpv[t] - np.sum(ypv[:, t])
    delta_BMm[t] = bmv[t] - np.sum(ymv[:, t])
    total_surplus[t]   = P_DA[t] * delta_DA[t] + P_RT[t] * delta_BMp[t] - P_PN[t] * delta_BMm[t]

print(f"{'t':>2} | {'ΔDA':>10} | {'ΔBM+':>12} | {'ΔBM-':>12} || {'Surplus':>10}")
print("-" * 60)
for t in range(T):
    print(f"{t:2} | {delta_DA[t]:10.4f} | {delta_BMp[t]:12.4f} | {delta_BMm[t]:12.4f} || {total_surplus[t]:10.4f}")
print("-" * 60)
print(f"{'Σ':>2} | {'':>10} | {'':>12} | {'':>12} || {total_surplus.sum():10.4f}")

 t |        ΔDA |         ΔBM+ |         ΔBM- ||    Surplus
------------------------------------------------------------
 0 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 1 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 2 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 3 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 4 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 5 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 6 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 7 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 8 |     1.0643 |      -1.0643 |       0.0000 ||    20.7502
 9 |     1.7165 |      -1.7165 |       0.0000 ||     9.3926
10 |     3.1488 |      -3.1488 |       0.0000 ||    50.6496
11 |    35.0264 |     -35.0264 |       0.0000 ||   436.5609
12 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
13 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
14 |   145.6823 |    -145.6823 |       

In [485]:
Po        = xv + ypv - ymv                         
Po_prime  = xv + ypv - ymv
delta_Po  = Po - Po_prime
delta_Pop = np.where(delta_Po > 0,  delta_Po, 0)
delta_Pom = np.where(delta_Po < 0, -delta_Po, 0)

print(f"{'i':>2} {'t':>2} | {'Po':>10} | {'Po′':>10} | {'ΔPo':>10} | {'ΔP+':>10} | {'ΔP-':>10}")
print("-" * 70)
for i in range(I):
    for t in range(9, 12):
        print(f"{i:2} {t:2} | {Po[i, t]:10.2f} | {Po_prime[i, t]:10.2f} | {delta_Po[i, t]:10.2f} | {delta_Pop[i, t]:10.2f} | {delta_Pom[i, t]:10.2f}")

 i  t |         Po |        Po′ |        ΔPo |        ΔP+ |        ΔP-
----------------------------------------------------------------------
 0  9 |       2.23 |       2.23 |       0.00 |       0.00 |       0.00
 0 10 |      22.81 |      22.81 |       0.00 |       0.00 |       0.00
 0 11 |      72.29 |      72.29 |       0.00 |       0.00 |       0.00
 1  9 |      21.35 |      21.35 |       0.00 |       0.00 |       0.00
 1 10 |      45.06 |      45.06 |       0.00 |       0.00 |       0.00
 1 11 |      24.64 |      24.64 |       0.00 |       0.00 |       0.00


In [486]:
varphi_p = ypv + delta_Pop
varphi_m = ymv + delta_Pom

print(f"{'i':>2} {'t':>2} | {'y⁺':>8} | {'ΔP⁺':>8} | {'φ⁺':>8} || {'y⁻':>8} | {'ΔP⁻':>8} | {'φ⁻':>8}")
print("-" * 75)
for i in range(I):
    for t in range(T):
        print(f"{i:2} {t:2} | {ypv[i,t]:8.2f} | {delta_Pop[i,t]:8.2f} | {varphi_p[i,t]:8.2f} || {ymv[i,t]:8.2f} | {delta_Pom[i,t]:8.2f} | {varphi_m[i,t]:8.2f}")

 i  t |       y⁺ |      ΔP⁺ |       φ⁺ ||       y⁻ |      ΔP⁻ |       φ⁻
---------------------------------------------------------------------------
 0  0 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  1 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  2 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  3 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  4 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  5 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  6 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  7 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  8 |    14.35 |     0.00 |    14.35 ||     0.00 |     0.00 |     0.00
 0  9 |     0.85 |     0.00 |     0.85 ||     0.00 |     0.00 |     0.00
 0 10 |     3.15 |     0.00 |     3.15 ||     0.00 |     0.00 |     0.00
 0 11 |    54.24 |     0.00 |    54.24 ||     0.

In [487]:
delta_DAp = np.where(delta_DA > 0,  delta_DA, 0)
delta_DAm = np.where(delta_DA < 0, -delta_DA, 0)
print(f"{'t':>2} | {'α':>6} | {'∑x':>6} | {'ΔDA':>8} | {'ΔDA+':>8} | {'ΔDA-':>8}")
print("-" * 55)
for t in range(T):
    print(f"{t:2} | {av[t]:6.2f} | {np.sum(xv[:,t]):6.2f} | {delta_DA[t]:8.2f} | {delta_DAp[t]:8.2f} | {delta_DAm[t]:8.2f}")

 t |      α |     ∑x |      ΔDA |     ΔDA+ |     ΔDA-
-------------------------------------------------------
 0 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 1 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 2 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 3 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 4 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 5 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 6 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 7 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 8 |  13.52 |  12.46 |     1.06 |     1.06 |     0.00
 9 |   7.29 |   5.57 |     1.72 |     1.72 |     0.00
10 |  67.87 |  64.72 |     3.15 |     3.15 |     0.00
11 |  77.72 |  42.69 |    35.03 |    35.03 |     0.00
12 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
13 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
14 | 349.73 | 204.05 |   145.68 |   145.68 |     0.00
15 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
16 |  70.66 |  64.52 |    

In [488]:
E_INp = np.zeros(T)
E_INm = np.zeros(T)

for t in range(T):
    E_INp[t] = np.sum(varphi_p[:, t]) - (bpv[t] + delta_DAp[t])
    E_INm[t] = np.sum(varphi_m[:, t]) - (bmv[t] + delta_DAm[t])
    
print(f"{'t':>2} | {'∑φ⁺':>8} | {'β⁺':>6} | {'ΔDA+':>6} | {'E_IN⁺':>8} || {'∑φ⁻':>8} | {'β⁻':>6} | {'ΔDA-':>6} | {'E_IN⁻':>8}")
print("-" * 90)
for t in range(T):
    sum_phi_p = np.sum(varphi_p[:, t])
    sum_phi_m = np.sum(varphi_m[:, t])
    print(f"{t:2} | {sum_phi_p:8.2f} | {bpv[t]:6.2f} | {delta_DAp[t]:6.2f} | {E_INp[t]:8.2f} ||"
          f" {sum_phi_m:8.2f} | {bmv[t]:6.2f} | {delta_DAm[t]:6.2f} | {E_INm[t]:8.2f}")

 t |      ∑φ⁺ |     β⁺ |   ΔDA+ |    E_IN⁺ ||      ∑φ⁻ |     β⁻ |   ΔDA- |    E_IN⁻
------------------------------------------------------------------------------------------
 0 |     0.00 |   0.00 |   0.00 |     0.00 ||     0.00 |   0.00 |   0.00 |     0.00
 1 |     0.00 |   0.00 |   0.00 |     0.00 ||     0.00 |   0.00 |   0.00 |     0.00
 2 |     0.00 |   0.00 |   0.00 |     0.00 ||     0.00 |   0.00 |   0.00 |     0.00
 3 |     0.00 |   0.00 |   0.00 |     0.00 ||     0.00 |   0.00 |   0.00 |     0.00
 4 |     0.00 |   0.00 |   0.00 |     0.00 ||     0.00 |   0.00 |   0.00 |     0.00
 5 |     0.00 |   0.00 |   0.00 |     0.00 ||     0.00 |   0.00 |   0.00 |     0.00
 6 |     0.00 |   0.00 |   0.00 |     0.00 ||     0.00 |   0.00 |   0.00 |     0.00
 7 |     5.69 |   5.69 |   0.00 |     0.00 ||     0.00 |   0.00 |   0.00 |     0.00
 8 |    16.00 |  14.93 |   1.06 |     0.00 ||     0.00 |   0.00 |   0.00 |     0.00
 9 |    18.01 |  16.30 |   1.72 |     0.00 ||     0.00 |   0.00 |   0

In [489]:
m = 1e-16
e_INp = np.zeros((I, T))
e_INm = np.zeros((I, T))

for t in range(T):
    for i in range(I):
        e_INp[i, t] = (varphi_p[i, t] / (np.sum(varphi_p[:, t]) + m)) * E_INp[t]
        e_INm[i, t] = (varphi_m[i, t] / (np.sum(varphi_m[:, t]) + m)) * E_INm[t]

print(f"{'i':>2} {'t':>2} | {'φ⁺':>8} | {'∑φ⁺':>8} | {'e_IN⁺':>10} || {'φ⁻':>8} | {'∑φ⁻':>8} | {'e_IN⁻':>10}")
print("-" * 80)
for t in range(T):
    for i in range(I):
        sum_phi_p = np.sum(varphi_p[:, t])
        sum_phi_m = np.sum(varphi_m[:, t])
        print(f"{i:2} {t:2} | {varphi_p[i, t]:8.2f} | {sum_phi_p:8.2f} | {e_INp[i, t]:10.2f} ||"
              f" {varphi_m[i, t]:8.2f} | {sum_phi_m:8.2f} | {e_INm[i, t]:10.2f}")

 i  t |       φ⁺ |      ∑φ⁺ |      e_IN⁺ ||       φ⁻ |      ∑φ⁻ |      e_IN⁻
--------------------------------------------------------------------------------
 0  0 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 1  0 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 0  1 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 1  1 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 0  2 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 1  2 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 0  3 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 1  3 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 0  4 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 1  4 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 0  5 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       

In [490]:
delta_INp = np.zeros(T)
delta_INm = np.zeros(T)
delta_SCp = np.zeros(T)
delta_SCm = np.zeros(T)
Lambda_IN = np.zeros(T)
rho_IN = np.zeros(T)

for t in range(T):
    # 조건: BM+ < 0, BM- < 0
    if delta_BMp[t] < 0:
        delta_INp[t] = -min(E_INp[t], -delta_BMp[t])
    else:
        delta_INp[t] = 0

    if delta_BMm[t] < 0:
        delta_INm[t] = -min(E_INm[t], -delta_BMm[t])
    else:
        delta_INm[t] = 0
    
    delta_SCp[t] = delta_BMp[t] - delta_INp[t]
    delta_SCm[t] = delta_BMm[t] - delta_INm[t]

    Lambda_IN[t] = P_RT[t] * delta_INp[t] - P_PN[t] * delta_INm[t]

    # if total_IN < 1e-6:
    #     rho_IN[t] = 0
    #     Lambda_IN[t] = 0
    # else:
    #     rho_IN[t] = Lambda_IN[t] / (total_IN + m)
    rho_IN[t] = Lambda_IN[t] / (E_INp[t] + E_INm[t] + m)
    
print(f"{'t':>2} | {'ΔBM+':>7} | {'E_IN+':>7} | {'ΔIN+':>7} | {'ΔSC+':>7} || "
      f"{'ΔBM-':>7} | {'E_IN-':>7} | {'ΔIN-':>7} | {'ΔSC-':>7} || "
      f"{'Λ_IN':>8} | {'ρ_IN':>8}")
print("-" * 110)

for t in range(T):
    print(f"{t:2} | {delta_BMp[t]:7.2f} | {E_INp[t]:7.2f} | {delta_INp[t]:7.2f} | {delta_SCp[t]:7.2f} || "
          f"{delta_BMm[t]:7.2f} | {E_INm[t]:7.2f} | {delta_INm[t]:7.2f} | {delta_SCm[t]:7.2f} || "
          f"{Lambda_IN[t]:8.2f} | {rho_IN[t]:8.2f}")

 t |    ΔBM+ |   E_IN+ |    ΔIN+ |    ΔSC+ ||    ΔBM- |   E_IN- |    ΔIN- |    ΔSC- ||     Λ_IN |     ρ_IN
--------------------------------------------------------------------------------------------------------------
 0 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 1 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 2 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 3 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 4 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 5 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 6 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 7 |    0.00 |    0.00 |    0.00 

In [499]:
e_SCp = np.zeros((I, T))
e_SCm = np.zeros((I, T))
delta_SCpp = np.where(delta_SCp > 0,  delta_SCp, 0)
delta_SCpm = np.where(delta_SCp < 0, -delta_SCp, 0)
delta_SCmp = np.where(delta_SCm > 0,  delta_SCm, 0)
delta_SCmm = np.where(delta_SCm < 0, -delta_SCm, 0)
Lambda_d = np.zeros(T)
rho_d = np.zeros(T)

for t in range(T):
    mp = np.sum(varphi_p[:, t] - e_INp[:, t]) + m
    mm = np.sum(varphi_m[:, t] - e_INm[:, t]) + m
    e_SCp[:, t] = (varphi_p[:, t] - e_INp[:, t]) / mp * (delta_DAp[t] - (delta_SCpp[t] + delta_SCmm[t]))
    e_SCm[:, t] = (varphi_m[:, t] - e_INm[:, t]) / mm * (delta_DAm[t] - (delta_SCpm[t] + delta_SCmp[t]))
    
    Lambda_d[t] = P_DA[t] * (delta_DA[t] + delta_SCp[t] - delta_SCm[t])
    rho_d[t] = Lambda_d[t] / (np.sum(e_SCp[:, t] + e_SCm[:, t]) + m)

print(f"{'i':>2} {'t':>2} | {'φ⁺-eIN⁺':>10} | {'∑':>10} | {'ΔDA+':>7} | {'SC++ + SC--':>14} | {'e_SC+':>9} || "
      f"{'φ⁻-eIN⁻':>10} | {'∑':>10} | {'ΔDA-':>7} | {'SC+- + SC-+':>14} | {'e_SC-':>9} || "
      f"{'Λ_d':>8} | {'ρ_d':>8}")
print("-" * 165)

for i in range(I):
    for t in range(T):
        num_p = varphi_p[i, t] - e_INp[i, t]
        num_m = varphi_m[i, t] - e_INm[i, t]
        sum_p = np.sum(varphi_p[:, t] - e_INp[:, t])
        sum_m = np.sum(varphi_m[:, t] - e_INm[:, t])
        SCpp_plus_SCmm = delta_SCpp[t] + delta_SCmm[t]
        SCpm_plus_SCmp = delta_SCpm[t] + delta_SCmp[t]

        print(f"{i:2} {t:2} | {num_p:10.4f} | {sum_p:10.4f} | {delta_DAp[t]:7.2f} | {SCpp_plus_SCmm:14.2f} | {e_SCp[i, t]:9.2f} || "
              f"{num_m:10.2f} | {sum_m:10.2f} | {delta_DAm[t]:7.2f} | {SCpm_plus_SCmp:14.2f} | {e_SCm[i, t]:9.2f} || "
              f"{Lambda_d[t]:8.2f} | {rho_d[t]:8.2f}")

 i  t |    φ⁺-eIN⁺ |          ∑ |    ΔDA+ |    SC++ + SC-- |     e_SC+ ||    φ⁻-eIN⁻ |          ∑ |    ΔDA- |    SC+- + SC-+ |     e_SC- ||      Λ_d |      ρ_d
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 0  0 |     0.0000 |     0.0000 |    0.00 |           0.00 |      0.00 ||       0.00 |       0.00 |    0.00 |           0.00 |      0.00 ||     0.00 |     0.00
 0  1 |     0.0000 |     0.0000 |    0.00 |           0.00 |      0.00 ||       0.00 |       0.00 |    0.00 |           0.00 |      0.00 ||     0.00 |     0.00
 0  2 |     0.0000 |     0.0000 |    0.00 |           0.00 |      0.00 ||       0.00 |       0.00 |    0.00 |           0.00 |      0.00 ||     0.00 |     0.00
 0  3 |     0.0000 |     0.0000 |    0.00 |           0.00 |      0.00 ||       0.00 |       0.00 |    0.00 |           0.00 |      0.00 ||     0.00 |     0.00
 0  4 |     0.0000 |     0.0000 | 

In [526]:
Lambda_b = np.zeros(T)
rho_b = np.zeros(T)
e_bp = np.zeros((I, T))
e_bm = np.zeros((I, T))

for t in range(T):
    Lambda_b[t] = (P_RT[t] - P_DA[t]) * delta_SCp[t] - (P_PN[t] - P_DA[t]) * delta_SCm[t]
    
    e_bp[:, t] = varphi_p[:, t] - e_INp[:, t] - e_SCp[:, t]
    e_bm[:, t] = varphi_m[:, t] - e_INm[:, t] - e_SCm[:, t]


    # # 만약 거래가 거의 없다면, rho_b는 0으로 처리
    # if total_b < 1e-6:
    #     rho_b[t] = 0
    #     Lambda_b[t] = 0
    # else:
    #     rho_b[t] = Lambda_b[t] / (total_b + m)
    rho_b[t] = Lambda_b[t] / (np.sum(e_bp[:, t] + e_bm[:, t]) + m)
    
print(f"{'i':>2} {'t':>2} | {'φ⁺':>7} | {'e_IN⁺':>7} | {'e_SC⁺':>7} | {'e_b+':>9} || "
      f"{'φ⁻':>7} | {'e_IN⁻':>7} | {'e_SC⁻':>7} | {'e_b-':>9} || {'Λ_b':>10} | {'ρ_b':>8}")
print("-" * 120)

for i in range(I):
    for t in range(9, 12):
        print(f"{i:2} {t:2} | "
              f"{varphi_p[i, t]:7.4f} | {e_INp[i, t]:7.4f} | {e_SCp[i, t]:7.4f} | {e_bp[i, t]:9.4f} || "
              f"{varphi_m[i, t]:7.4f} | {e_INm[i, t]:7.4f} | {e_SCm[i, t]:7.4f} | {e_bm[i, t]:9.4f} || "
              f"{Lambda_b[t]:10.2f} | {rho_b[t]:8.2f}")

 i  t |      φ⁺ |   e_IN⁺ |   e_SC⁺ |      e_b+ ||      φ⁻ |   e_IN⁻ |   e_SC⁻ |      e_b- ||        Λ_b |      ρ_b
------------------------------------------------------------------------------------------------------------------------
 0  9 |  0.8518 |  0.0000 |  0.0812 |    0.7707 ||  0.0000 |  0.0000 | -0.0000 |    0.0000 ||       9.39 |     0.58
 0 10 |  3.1488 |  0.0000 |  3.1488 |    0.0000 ||  0.0000 |  0.0000 | -0.0000 |    0.0000 ||      50.65 | 506496188417301376.00
 0 11 | 54.2374 |  0.0000 | 35.0264 |   19.2111 ||  0.0000 |  0.0000 | -0.0000 |    0.0000 ||     436.56 |    22.72
 1  9 | 17.1624 |  0.0000 |  1.6353 |   15.5271 ||  0.0000 |  0.0000 | -0.0000 |    0.0000 ||       9.39 |     0.58
 1 10 |  0.0000 |  0.0000 |  0.0000 |    0.0000 ||  0.0000 |  0.0000 | -0.0000 |    0.0000 ||      50.65 | 506496188417301376.00
 1 11 |  0.0000 |  0.0000 |  0.0000 |    0.0000 ||  0.0000 |  0.0000 | -0.0000 |    0.0000 ||     436.56 |    22.72


In [527]:
lambda_it = np.zeros((I, T))
lambda_sum = np.zeros(T)

for t in range(T):
    for i in range(I):
        if rho_IN[t] > 0 and (e_INp[i, t] + e_INm[i, t]) > 1e-6:
            lambda_it[i, t] += rho_IN[t] * (e_INp[i, t] + e_INm[i, t])
        if rho_d[t] > 0 and (e_SCp[i, t] + e_SCm[i, t]) > 1e-6:
            lambda_it[i, t] += rho_d[t] * (e_SCp[i, t] + e_SCm[i, t])
        if rho_b[t] > 0 and (e_bp[i, t] + e_bm[i, t]) > 1e-6:
            lambda_it[i, t] += rho_b[t] * (e_bp[i, t] + e_bm[i, t])
    lambda_sum[t] = np.sum(lambda_it[:, t])     

print(f"{'i':>2} {'t':>2} | {'λ_it':>10}")
print("-" * 20)
for t, i in product(range(T), range(I)):
    print(f"{i:2} {t:2} | {lambda_it[i, t]:10.4f}")

 i  t |       λ_it
--------------------
 0  0 |     0.0000
 1  0 |     0.0000
 0  1 |     0.0000
 1  1 |     0.0000
 0  2 |     0.0000
 1  2 |     0.0000
 0  3 |     0.0000
 1  3 |     0.0000
 0  4 |     0.0000
 1  4 |     0.0000
 0  5 |     0.0000
 1  5 |     0.0000
 0  6 |     0.0000
 1  6 |     0.0000
 0  7 |     0.0000
 1  7 |     0.0000
 0  8 |    18.6107
 1  8 |     2.1395
 0  9 |     0.4441
 1  9 |     8.9484
 0 10 |     0.0000
 1 10 |     0.0000
 0 11 |   436.5609
 1 11 |     0.0000
 0 12 |     0.0000
 1 12 |     0.0000
 0 13 |     0.0000
 1 13 |     0.0000
 0 14 |  3868.7802
 1 14 |  2857.8772
 0 15 |     0.0000
 1 15 |     0.0000
 0 16 |     0.0000
 1 16 |     0.0000
 0 17 |     0.0000
 1 17 |     0.0000
 0 18 |     0.0000
 1 18 |     0.0000
 0 19 |   857.6199
 1 19 |   857.6199
 0 20 |     0.0000
 1 20 |     0.0000
 0 21 |     0.0000
 1 21 |     0.0000
 0 22 |     0.0000
 1 22 |     0.0000
 0 23 |     0.0000
 1 23 |     0.0000


In [528]:
print(f"{'t':>2} | {'Λ_IN':>8} | {'Λ_d':>8} | {'Λ_b':>8} || {'∑λ_it':>10} | {'Difference':>10}")
print("-" * 65)

total_IN = 0
total_d  = 0
total_b  = 0
total_lambda = 0

for t in range(T):
    total_lhs = Lambda_IN[t] + Lambda_d[t] + Lambda_b[t]
    diff = total_lhs - lambda_sum[t]

    # 누적합 계산
    total_IN     += Lambda_IN[t]
    total_d      += Lambda_d[t]
    total_b      += Lambda_b[t]
    total_lambda += lambda_sum[t]

    print(f"{t:2} | {Lambda_IN[t]:8.2f} | {Lambda_d[t]:8.2f} | {Lambda_b[t]:8.2f} || "
          f"{lambda_sum[t]:10.2f} | {diff:10.2f}")

# 마지막 줄: 총합 정리
print("-" * 65)
print(f"{'Σ':>2} | {total_IN:8.2f} | {total_d:8.2f} | {total_b:8.2f} || "
      f"{total_lambda:10.2f} | {total_IN + total_d + total_b - total_lambda:10.2f}")

 t |     Λ_IN |      Λ_d |      Λ_b ||      ∑λ_it | Difference
-----------------------------------------------------------------
 0 |     0.00 |     0.00 |    -0.00 ||       0.00 |       0.00
 1 |     0.00 |     0.00 |     0.00 ||       0.00 |       0.00
 2 |     0.00 |     0.00 |     0.00 ||       0.00 |       0.00
 3 |     0.00 |     0.00 |    -0.00 ||       0.00 |       0.00
 4 |     0.00 |     0.00 |    -0.00 ||       0.00 |       0.00
 5 |     0.00 |     0.00 |    -0.00 ||       0.00 |       0.00
 6 |     0.00 |     0.00 |    -0.00 ||       0.00 |       0.00
 7 |     0.00 |     0.00 |    -0.00 ||       0.00 |       0.00
 8 |    -0.00 |     0.00 |    20.75 ||      20.75 |       0.00
 9 |    -0.00 |     0.00 |     9.39 ||       9.39 |       0.00
10 |    -0.00 |     0.00 |    50.65 ||       0.00 |      50.65
11 |    -0.00 |     0.00 |   436.56 ||     436.56 |      -0.00
12 |     0.00 |     0.00 |     0.00 ||       0.00 |       0.00
13 |     0.00 |     0.00 |     0.00 ||       0.00 | 